This implementation employs the successive training over the given dataset which does not utilize the randomness (hence, shuffling technique) in training the network

The remarks can be found in this implementation can be interpreted as:

In [1]:
import torch
from torch.autograd import grad
import torch.nn as nn
import numpy as np
import math
import torch.optim as optim

import h5py
import scipy.io

torch.set_default_dtype(torch.float32)

device = 'cpu' if torch.cuda.is_available() else 'cpu'

In [2]:
class FCNNs(nn.Module):
    def __init__(self, input_dim = 1, output_dim = 3, hidden_layer_dim = 128, num_hidden_layer = 5, activation = 'tanh'):

        super(FCNNs, self).__init__()

        self.layers = nn.ModuleList([nn.Linear(input_dim, hidden_layer_dim)])
        for _ in range(num_hidden_layer - 1):
            self.layers.append(nn.Linear(hidden_layer_dim, hidden_layer_dim))
        self.layers.append(nn.Linear(hidden_layer_dim, output_dim))

        self.epoch = 0
        
        if activation == 'sin':
          self.activation = torch.sin
        elif activation == 'tanh':
          self.activation = torch.tanh
        elif activation == 'relu':
          self.activation = torch.nn.ReLU()
        else:
          self.activation = torch.nn.LeakyReLU()

    def forward(self, t):

        out = t
        for layer in self.layers[:-1]:
            out = self.activation(layer(out))
        out = self.layers[-1](out)
        return out

In [ ]:
class Delta_Robot_Dynamic(FCNNs):
    def __init__(self, t, tau, s, s_Ddot, u_data):
        super(Delta_Robot_Dynamic, self).__init__()
        
        #-------------Initialize base parameters-----------#
        self.f = 0.06
        self.e = 0.045
        self.l_1 = 0.176
        self.alpha = torch.deg2rad(torch.tensor([-30, 90, 210], dtype = torch.float32))
        
        self.g = 9.8
        
        #-------------Initialize learnable parameters--------------#
        self.l_1c = torch.tensor([0.01], requires_grad=True, device = device)
        self.m_0 = torch.tensor([0.01], requires_grad=True, device = device)
        self.m_2 = torch.tensor([0.01], requires_grad=True, device = device)
        self.m_1 = torch.tensor([0.01], requires_grad=True, device = device)
        self.I_1 = torch.tensor([0.01], requires_grad=True, device = device)
        
        self.f_v1 = torch.tensor([0.05], requires_grad=True, device = device)
        self.f_v2 = torch.tensor([0.05], requires_grad=True, device = device)
        self.f_v3 = torch.tensor([0.05], requires_grad=True, device = device)
        
        self.f_c1 = torch.tensor([0.05], requires_grad=True, device = device)
        self.f_c2 = torch.tensor([0.05], requires_grad=True, device = device)
        self.f_c3 = torch.tensor([0.05], requires_grad=True, device = device)
        
        #--------------Convert acquired data-------------#
        self.t = t.to(device).detach().requires_grad_(True)
        
        self.tau = tau
        self.tau_1 = tau[:, 0:1].to(device)
        self.tau_2 = tau[:, 1:2].to(device)
        self.tau_3 = tau[:, 2:3].to(device)
        
        self.s = s.to(device)
        
        self.s_Ddot = s_Ddot.to(device)
        
        self.theta_1 = u_data[:, 0:1].to(device)
        self.theta_2 = u_data[:, 1:2].to(device)
        self.theta_3 = u_data[:, 2:3].to(device)
                        
        #---------------Import FCNNs model---------------#
        self.model = FCNNs().to(device)
        
        #-------Implement 2-stage Optimizer--------------#
        self.optimizer_Adam = optim.Adam([
            {'params': self.model.parameters()},
            {'params': [self.f_v1, self.f_v2, self.f_v3]},
            {'params': [self.f_c1, self.f_c2, self.f_c3]},
            {'params': [self.m_0, self.m_1, self.m_2]},
            {'params': [self.l_1c, self.I_1]}
        ], lr=1e-3, betas=(0.9, 0.999), eps=1e-08)
    
        
    def compute_derivatives(self, t, x_pred):
        
        x_pred_grad = torch.autograd.grad(
            outputs=x_pred, inputs=t,
            grad_outputs=torch.ones_like(x_pred),
            create_graph=True
        )[0]
        
        x_pred_grad2 = torch.autograd.grad(
            outputs=x_pred_grad, inputs=t,
            grad_outputs=torch.ones_like(x_pred_grad),
            create_graph=True
        )[0]

        return x_pred_grad, x_pred_grad2
    
    def Inverse_Dynamic_eq(self):
        
        self.theta_1_pred = self.model(self.t)[:, 0:1]                                                  # shape: batch_size x 1
        self.theta_2_pred = self.model(self.t)[:, 1:2]                                                  # shape: batch_size x 1
        self.theta_3_pred = self.model(self.t)[:, 2:3]                                                  # shape: batch_size x 1
                
        
        theta_grad_1, theta_grad2_1 = self.compute_derivatives(self.t, self.theta_1_pred)               # shape: batch_size x 1
        theta_grad_2, theta_grad2_2 = self.compute_derivatives(self.t, self.theta_2_pred)               # shape: batch_size x 1
        theta_grad_3, theta_grad2_3 = self.compute_derivatives(self.t, self.theta_3_pred)               # shape: batch_size x 1
                
        theta_dot  = torch.cat([theta_grad_1, theta_grad_2, theta_grad_3], dim = 1)                     # shape: batch_size x 3
        theta_Ddot = torch.cat([theta_grad2_1, theta_grad2_2, theta_grad2_3], dim = 1)                  # shape: batch_size x 3
                
        #---------------Define M matrix----------------#
        M = torch.zeros(self.theta_1.shape[0], 3, 3)
        M[:, 0, 0] = 2 * self.I_1 + self.m_2 * (self.l_1 ** 2)
        M[:, 1, 1] = 2 * self.I_1 + self.m_2 * (self.l_1 ** 2)
        M[:, 2, 2] = 2 * self.I_1 + self.m_2 * (self.l_1 ** 2)
                
        #---------------Define G matrix-----------------#
        G = torch.zeros(self.theta_1.shape[0], 3, 1)
        G[:, 0, 0] = (self.m_1 * self.l_1c + self.m_2 * self.l_1) * self.g * torch.cos(self.theta_1_pred.squeeze(-1))             
        G[:, 1, 0] = (self.m_1 * self.l_1c + self.m_2 * self.l_1) * self.g * torch.cos(self.theta_2_pred.squeeze(-1))                                    
        G[:, 2, 0] = (self.m_1 * self.l_1c + self.m_2 * self.l_1) * self.g * torch.cos(self.theta_3_pred.squeeze(-1))                                     
                
        #---------------------------Define Friction matrices----------------------------#
        #-----Create friction matrices for F_v and F_c with shape [batch_size, 3, 3]----#
        F_v = torch.zeros(3, 3)
        F_v[0, 0] = self.f_v1
        F_v[1, 1] = self.f_v2
        F_v[2, 2] = self.f_v3
        
        F_v = F_v.unsqueeze(0).expand(self.theta_1.shape[0], -1, -1)                                               
        
        
        F_c = torch.zeros(3, 3)
        F_c[0, 0] = self.f_c1
        F_c[1, 1] = self.f_c2
        F_c[2, 2] = self.f_c3
        
        F_c = F_c.unsqueeze(0).expand(self.theta_1.shape[0], -1, -1) 
        
        
        #----------------Define K matrix---------------------#
        K11 = (self.s[:, 0] * torch.cos((self.alpha[0])) + self.s[:, 1] * torch.sin((self.alpha[0])) + self.f - self.e) * torch.sin(self.theta_1_pred.squeeze(-1)) - self.s[:, 2] * torch.cos(self.theta_1_pred.squeeze(-1))
        K22 = (self.s[:, 0] * torch.cos((self.alpha[1])) + self.s[:, 1] * torch.sin((self.alpha[1])) + self.f - self.e) * torch.sin(self.theta_2_pred.squeeze(-1)) - self.s[:, 2] * torch.cos(self.theta_2_pred.squeeze(-1))
        K33 = (self.s[:, 0] * torch.cos((self.alpha[2])) + self.s[:, 1] * torch.sin((self.alpha[2])) + self.f - self.e) * torch.sin(self.theta_3_pred.squeeze(-1)) - self.s[:, 2] * torch.cos(self.theta_3_pred.squeeze(-1))
        
        K = torch.zeros((self.theta_1_pred.shape[0], 3, 3))                                                            
        K[:, 0, 0] = K11
        K[:, 1, 1] = K22
        K[:, 2, 2] = K33
        
        #--------------------------------Define A matrix--------------------------------#
        A = torch.zeros((self.theta_1_pred.shape[0], 3, 3))                                                             # batch_size x 3 x 3
        
        A[:, 0, 0] = self.s[:, 0] + self.e * torch.cos((self.alpha[0])) - self.f * torch.cos((self.alpha[0])) - self.l_1 * torch.cos((self.alpha[0])) * torch.cos(self.theta_1_pred.squeeze(-1)) # shape: batch_size
        A[:, 0, 1] = self.s[:, 0] + self.e * torch.cos((self.alpha[1])) - self.f * torch.cos((self.alpha[1])) - self.l_1 * torch.cos((self.alpha[1])) * torch.cos(self.theta_2_pred.squeeze(-1)) # shape: batch_size
        A[:, 0, 2] = self.s[:, 0] + self.e * torch.cos((self.alpha[2])) - self.f * torch.cos((self.alpha[2])) - self.l_1 * torch.cos((self.alpha[2])) * torch.cos(self.theta_3_pred.squeeze(-1)) # shape: batch_size

        A[:, 1, 0] = self.s[:, 1] + self.e * torch.sin((self.alpha[0])) - self.f * torch.sin((self.alpha[0])) - self.l_1 * torch.sin((self.alpha[0])) * torch.cos(self.theta_1_pred.squeeze(-1)) # shape: batch_size
        A[:, 1, 1] = self.s[:, 1] + self.e * torch.sin((self.alpha[1])) - self.f * torch.sin((self.alpha[1])) - self.l_1 * torch.sin((self.alpha[1])) * torch.cos(self.theta_2_pred.squeeze(-1)) # shape: batch_size
        A[:, 1, 2] = self.s[:, 1] + self.e * torch.sin((self.alpha[2])) - self.f * torch.sin((self.alpha[2])) - self.l_1 * torch.sin((self.alpha[2])) * torch.cos(self.theta_3_pred.squeeze(-1)) # shape: batch_size

        A[:, 2, 0] = self.s[:, 2] - self.l_1 * torch.cos(self.theta_1_pred.squeeze(-1)) # shape: batch_size
        A[:, 2, 1] = self.s[:, 2] - self.l_1 * torch.cos(self.theta_2_pred.squeeze(-1)) # shape: batch_size
        A[:, 2, 2] = self.s[:, 2] - self.l_1 * torch.cos(self.theta_3_pred.squeeze(-1)) # shape: batch_size
        
        
        #------------Define B matrix-----------#
        B = torch.zeros((self.theta_1.shape[0], 3, 1))                                                            # batch_size x 3 x 1
        
        B[:, 0, 0] = (self.m_0 + 3 * self.m_2) * self.s_Ddot[:, 0]
        B[:, 1, 0] = (self.m_0 + 3 * self.m_2) * self.s_Ddot[:, 1]
        B[:, 2, 0] = (self.m_0 + 3 * self.m_2) * (self.s_Ddot[:, 2] - self.g)
        
        equation = torch.matmul(M, theta_Ddot.unsqueeze(-1)) + G - torch.matmul(torch.matmul(K, torch.linalg.inv(A)), B) + torch.matmul(F_v, (theta_dot.unsqueeze(-1))) + torch.matmul(F_c, (torch.sign(theta_dot.unsqueeze(-1)))) - self.tau.unsqueeze(-1)
        equation = equation.squeeze(-1)

        return equation
        
    def residual_loss(self):
        equation = self.Inverse_Dynamic_eq()
        return torch.mean(equation[:, 0:1] ** 2) + torch.mean(equation[:, 1:2] ** 2) + torch.mean(equation[:, 2:3] ** 2)
    
    def data_loss(self):
        
        return torch.mean((self.theta_1_pred - self.theta_1) ** 2) + torch.mean((self.theta_2_pred - self.theta_2) ** 2) + torch.mean((self.theta_3_pred - self.theta_3) ** 2)
    
    
    def train(self, iterations = 50000, changing_point = 2):
        for epoch in range(iterations * changing_point):
            self.optimizer_Adam.zero_grad()
            loss = 10 * self.residual_loss() + self.data_loss()
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 100 == 0:
                print(  f"Epoch {epoch:04d}, Loss: {loss.item():.6f}, \n" )            
                
                print(  f"f_v1: {self.f_v1.item(): .6f}, "
                        f"f_v2: {self.f_v2.item(): .6f}, "
                        f"f_v3: {self.f_v3.item(): .6f}, \n"
                    )
                
                print(  f"f_c1: {self.f_c1.item(): .6f}, "
                        f"f_c2: {self.f_c2.item(): .6f}, "
                        f"f_c3: {self.f_c3.item(): .6f}, \n"
                    )
                
                print(  f"m_0: {self.m_0.item(): .6f}, "
                        f"m_1: {self.m_1.item(): .6f}, "
                        f"m_2: {self.m_2.item(): .6f}, \n"
                    )
                
                print("|_________________________________| \n")
                
            self.epoch += 1
        


In [3]:
torch.manual_seed(0)
t = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_x.mat", "r")["data_x"][3:9998, 0]).unsqueeze(-1).float()


data_1 = torch.deg2rad(torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle1.mat", "r")["angle1"][3:9998, 1])).unsqueeze(1)
data_2 = torch.deg2rad(torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle2.mat", "r")["angle2"][3:9998, 1])).unsqueeze(1)
data_3 = torch.deg2rad(torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_angle3.mat", "r")["angle3"][3:9998, 1])).unsqueeze(1)
u_data = torch.cat([data_1, data_2, data_3], dim = 1)

tau_1 = torch.tensor(scipy.io.loadmat("C:/Users/FPTSHOP/Desktop/Project/tau_1.mat")["tau_first_row"][0, 3:9998]).unsqueeze(1) # I=2, Y=3 (ground truth)
tau_2 = torch.tensor(scipy.io.loadmat("C:/Users/FPTSHOP/Desktop/Project/tau_1.mat")["tau_second_row"][0, 3:9998]).unsqueeze(1)
tau_3 = torch.tensor(scipy.io.loadmat("C:/Users/FPTSHOP/Desktop/Project/tau_1.mat")["tau_third_row"][0, 3:9998]).unsqueeze(1)

tau = torch.cat([tau_1, tau_2, tau_3], dim = 1)

x_Ddot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_x_Ddot.mat", "r")["data_x_Ddot"][3:9998, 1]).unsqueeze(1) # 1996 x 1
y_Ddot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_y_Ddot.mat", "r")["data_y_Ddot"][3:9998, 1]).unsqueeze(1) # 1996 x 1
z_Ddot = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_z_Ddot.mat", "r")["data_z_Ddot"][3:9998, 1]).unsqueeze(1) # 1996 x 1

s_Ddot = torch.cat((x_Ddot, y_Ddot, z_Ddot), dim=1) * 0.001 

x = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_x.mat", "r")["data_x"][3:9998, 1]).unsqueeze(1)                # 1996 x 1
y = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_y.mat", "r")["data_y"][3:9998, 1]).unsqueeze(1)                # 1996 x 1
z = torch.tensor(h5py.File("C:/Users/FPTSHOP/Desktop/Project/simulated_z.mat", "r")["data_z"][3:9998, 1]).unsqueeze(1)                # 1996 x 1

s = torch.cat((x, y, z), dim=1) * 0.001   
 
print(s.size())

torch.Size([9995, 3])


In [ ]:
# Instantiate and train the PINN
pinn = Delta_Robot_Dynamic(t, tau, s, s_Ddot, u_data)
pinn.train()


Epoch 0000, Loss: 21.149888, 

f_v1:  0.051000, f_v2:  0.051000, f_v3:  0.051000, 

f_c1:  0.051000, f_c2:  0.051000, f_c3:  0.051000, 

m_0:  0.011000, m_1:  0.009000, m_2:  0.011000, 

|_________________________________| 

Epoch 0100, Loss: 0.965907, 

f_v1:  0.108313, f_v2: -0.004030, f_v3: -0.037103, 

f_c1:  0.033518, f_c2:  0.074228, f_c3: -0.018447, 

m_0:  0.079821, m_1: -0.000018, m_2:  0.079439, 

|_________________________________| 

Epoch 0200, Loss: 0.201138, 

f_v1:  0.109414, f_v2: -0.028350, f_v3: -0.062555, 

f_c1:  0.005925, f_c2:  0.036393, f_c3: -0.036816, 

m_0:  0.096208, m_1:  0.000001, m_2:  0.095716, 

|_________________________________| 

Epoch 0300, Loss: 0.086506, 

f_v1:  0.096199, f_v2: -0.046542, f_v3: -0.063806, 

f_c1: -0.008760, f_c2: -0.009476, f_c3: -0.036650, 

m_0:  0.102097, m_1:  0.000001, m_2:  0.102205, 

|_________________________________| 

Epoch 0400, Loss: 0.062927, 

f_v1:  0.080330, f_v2: -0.054281, f_v3: -0.058773, 

f_c1: -0.017762, f_c